In [1]:
!git init .
!git remote add origin https://github.com/kashparty/STP-GSR.git
!git pull origin discriminator

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 127 (delta 10), reused 8 (delta 4), pack-reused 102 (from 1)
Receiving objects: 100% (127/127), 64.80 MiB | 3.98 MiB/s, done.
Resolving deltas: 100% (53/53), done.
From https://github.com/kashparty/STP-GSR
 * branch            discriminator -> FETCH_HEAD
 * [new branch]      discriminator -> origin/discriminator
U

In [ ]:
import os
import hydra
import torch
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import KFold
from hydra import compose, initialize

from src.train import train, eval
from src.plot_utils import plot_adj_matrices
from src.dataset import load_dataset

def main():
    with initialize(version_base=None, config_path="configs"):
        config = compose(config_name="experiment")

    torch.cuda.empty_cache()

    if torch.cuda.is_available():
        print("Running on GPU")
    else:
        print("Running on CPU")

    kf = KFold(n_splits=config.experiment.kfold.n_splits,
               shuffle=config.experiment.kfold.shuffle,
               random_state=config.experiment.kfold.random_state)

    # Initialize folder structure for this run
    base_dir = config.experiment.base_dir
    model_name = config.model.name
    dataset_type = config.dataset.name
    run_name = config.experiment.run_name
    run_dir = f'{base_dir}/{model_name}/{dataset_type}/{run_name}/'

    # Load dataset
    source_data, target_data = load_dataset(config)

    for fold, (train_idx, val_idx) in enumerate(kf.split(source_data)):
        print(f"Training Fold {fold+1}/3")

        # Initialize results directory
        res_dir = f'{run_dir}fold_{fold+1}/'
        if not os.path.exists(res_dir):
            os.makedirs(res_dir)

        # Fetch training and val data for this fold
        source_data_train = [source_data[i] for i in train_idx]
        target_data_train = [target_data[i] for i in train_idx]
        source_data_val = [source_data[i] for i in val_idx]
        target_data_val = [target_data[i] for i in val_idx]

        # Train model for this fold
        train_output = train(config,
                              source_data_train,
                              target_data_train,
                              source_data_val,
                              target_data_val,
                              res_dir)

        # Evaluate model for this fold
        eval_output, eval_loss = eval(config,
                                      train_output['model'],
                                      source_data_val,
                                      target_data_val,
                                      train_output['critereon'])

        # Final evaluation loss for this fold
        print(f"Final Validation Loss (Target): {eval_loss}")

        # Save source, taregt, and eval output for this fold
        np.save(f'{res_dir}/eval_output.npy', np.array(eval_output))
        np.save(f'{res_dir}/source.npy', np.array([s['mat'] for s in source_data_val]))
        np.save(f'{res_dir}/target.npy', np.array([t['mat'] for t in target_data_val]))


        # Plot predictions for a random sample
        idx = 6
        source_mat_test = source_data_val[idx]['mat']
        target_mat_test = target_data_val[idx]['mat']
        eval_output_t = eval_output[idx]

        plot_adj_matrices(source_mat_test,
                          target_mat_test,
                          eval_output_t,
                          idx,
                          res_dir,
                          file_name=f'eval_sample{idx}')


main()

Running on CPU
Training Fold 1/3


  0%|          | 0/111 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (268x268 and 320x160)